# Create client machine on AWS

In [ ]:
## Settings

import os
import boto3

DF_HOST = "ec2-35-178-253-67.eu-west-2.compute.amazonaws.com"
CLUSTER_NAME = "demo.df.io"
TAG_PREFIX = "dfdemo"
KEY_PAIR_NAME = TAG_PREFIX + "-ez-keypair"
SECURITY_GROUP_NAME = TAG_PREFIX + "-ez-securitygroup"
INSTANCE_NAME = TAG_PREFIX + "-ez-node"
KEYFILE = TAG_PREFIX + "_key.pem"

ec2 = boto3.client('ec2')

response = ec2.describe_security_groups(GroupNames=[SECURITY_GROUP_NAME])
SECURITY_GROUP_ID = response.get("SecurityGroups", [{}])[0].get("GroupId", "")

print("Done")


In [ ]:
## Create AWS instance
response = ec2.run_instances(
    ImageId="ami-0bd2099338bc55e6d",
    MinCount=1,
    MaxCount=1,
    InstanceType="t2.micro",
    KeyName=KEY_PAIR_NAME,
    SecurityGroupIds=[
            SECURITY_GROUP_ID,
    ],
    BlockDeviceMappings=[
        {
            'DeviceName': "/dev/sda1",
            'Ebs': {
                'DeleteOnTermination': True,
                'VolumeSize': 30,
                'VolumeType': "gp2"
            }
        },
    ],
    TagSpecifications=[
        {
            'ResourceType': "instance",
            'Tags': [
                {
                    'Key': "Name",
                    'Value': TAG_PREFIX + "-client"
                },
            ]
        },
    ],
)
for i in response['Instances']:
    print("Waiting for instance %s to start..." % i['InstanceId'])
    ec2.get_waiter("instance_status_ok").wait(
        InstanceIds=[i['InstanceId']])
    INSTANCE_ID = i['InstanceId']

instance = ec2.describe_instances(InstanceIds=[INSTANCE_ID]).get(
    'Reservations', [{}])[0].get('Instances', '')[0]

print(instance['PublicDnsName'] + " is ready")

print("Done")


In [ ]:
# Allow connection from client to host

data = ec2.authorize_security_group_ingress(
    GroupId=SECURITY_GROUP_ID,
    IpPermissions=[
        {'IpProtocol': "-1",
         'IpRanges': [{'CidrIp': instance['PublicIpAddress']+'/32'}]}
    ])
print("Ingress Successfully Set for IP: %s" %
      data.get("SecurityGroupRules", [{}])[0].get('CidrIpv4', ''))


In [ ]:
# Helper functions

from paramiko import SSHClient, AutoAddPolicy, RSAKey

ip_address = instance['PublicIpAddress']
username = 'ubuntu'

def run_ssh_command(command):
  client = SSHClient()
  client.set_missing_host_key_policy(AutoAddPolicy())
  try:
      client.connect(ip_address, port=22,
                     username=username, key_filename=KEYFILE)

      print("Running", command)
      stdin, stdout, stderr = client.exec_command(command)
      for line in iter(stdout.readline, ""):
          yield line
      for line in iter(stderr.readline, ""):
          yield line

      stdin.close()
      stdout.close()
      stderr.close()
      client.close()

  except BaseException as e:
      print(e)


print("Done")


In [ ]:
# Get ssl_truststore
 
prvdns = instance['PrivateDnsName']
prvip = instance['PrivateIpAddress']
pubip = instance['PublicIpAddress']
pubdns = instance['PublicDnsName']

!echo $pubip $prvip $prvdns | sudo tee -a /etc/hosts > /dev/null

client = SSHClient()
client.set_missing_host_key_policy(AutoAddPolicy())
client.connect(ip_address, port=22,
               username='ubuntu', key_filename=KEYFILE)
sftp = client.open_sftp()

print("copy ssl_truststore")
sftp.put(localpath="./ssl_truststore", remotepath="/home/ubuntu/ssl_truststore")

sftp.close()
client.close()

print("Done")

In [ ]:
# Install client
commands = [
  'wget -O - https://package.mapr.hpe.com/releases/pub/maprgpg.key | sudo apt-key add -',
  'echo "deb https://package.mapr.hpe.com/releases/v7.0.0/ubuntu binary bionic" | sudo tee -a /etc/apt/sources.list',
  'echo "deb https://package.mapr.hpe.com/releases/MEP/MEP-8.1.0/ubuntu binary bionic" | sudo tee -a /etc/apt/sources.list',
  'DEBIAN_FRONTEND=noninteractive sudo apt update; sudo apt upgrade -y',
  'DEBIAN_FRONTEND=noninteractive sudo apt install -y openjdk-11-jdk',
  'DEBIAN_FRONTEND=noninteractive sudo apt install -y mapr-client',
  "[ -f /var/run/reboot-required ] && echo reboot and continue",
]
for command in commands:
  for line in run_ssh_command(command):
      print(line.rstrip('\n'))

print("Done")

In [ ]:
# Configure client

print('configuring for {} on {}'.format(CLUSTER_NAME, DF_HOST))

sedptrn = "'/%s/d'" % CLUSTER_NAME
commands = [
    'id mapr || (sudo groupadd -g 5000 mapr; sudo useradd -m -u 5000 -g 5000 -d /home/mapr -s /bin/bash mapr; sudo usermod -a -G sudo mapr)',
    'echo "mapr:mapr" | sudo chpasswd',
    '[ -d /mapr ] || sudo mkdir /mapr',
    '[ -d /data ] || sudo mkdir /data',
    'sed ' + sedptrn +
    ' /opt/mapr/conf/mapr-clusters.conf | sudo tee /opt/mapr/conf/mapr-clusters.conf',
    'sed ' + sedptrn +
    ' /etc/hosts | sudo tee /etc/hosts',
    'echo "{} {}" | sudo tee -a /etc/hosts'.format(
        instance['PublicIpAddress'], 
        CLUSTER_NAME),
    
  'sudo cp ssl_truststore /opt/mapr/conf/',
  'sudo /opt/mapr/server/configure.sh -c -N {} -C {}:7222 -HS {} -secure'.format(
      CLUSTER_NAME,
      DF_HOST,
      DF_HOST),
  'echo mapr | /opt/mapr/bin/maprlogin password -user mapr -cluster {}'.format(
      CLUSTER_NAME),
  # 'sudo /opt/mapr/hadoop/hadoop-2.7.6/bin/configure.sh --unsecure -EC "-HS {} --client"'.format(
  #       instance['PublicIpAddress'])
]

for command in commands:
  for line in run_ssh_command(command):
      print(line.rstrip('\n'))

print("Done")

### Client configured

In [ ]:
print(instance['PrivateIpAddress'])